In [15]:
# Impordid
import os
import csv
from datasets import load_dataset
from src.transformers import BertTokenizer, BertForMaskedLM, BertConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [7]:
input_folder = "korpus"
input_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename[-3:] == "tsv"]
input_files

['korpus\\eval.tsv']

In [8]:
# Võtab iga batchi (tekstid korpusest, kus laused on eraldatud \n sümboliga, 'text' : lause1 \n lause2 \n lause3)
# Tokenieerib laused ja lisab batchile labelid
# Väjund (x tähistab kõikide lausete arvu loetud batchi tekstides)
# {'input_ids': x*128*2 tensor, 'attention_mask': x*128 tensor, 'token_type_ids': x*128 tensor, 'labels': x*128*2 tensor}
def tokeniseeri_batch(batch):
    batch_laused = [lause for para in batch["text"] for lause in para.split("\n")]
    tokeniseeritud = tokenizer(batch_laused, max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")
    tokeniseeritud["labels"] = tokeniseeritud["input_ids"]
    return tokeniseeritud

# Tokeniseerija
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt", max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")

for filename in input_files[6:7]:
    print(filename)
    train_dataset = load_dataset("csv", data_files={'train': filename}, names = ["text"], delimiter = "\t")["train"]
    train_dataset_sisend = train_dataset.map(tokeniseeri_batch, batched=True, remove_columns=["text"])
    # train_dataset_sisend.set_format(type ='torch')
    out_name = filename[:-4] + ".json"
    train_dataset_sisend.to_json(out_name)
    

korpus\eval.tsv


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/rauln/.cache/huggingface/datasets/csv/default-6a711de5a5c10281/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

In [ ]:
%%time
train_dataset = load_dataset("csv", data_files={'train': input_files[6]}, names = ["text"], delimiter = "\t")["train"]

In [ ]:
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt", max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")

In [ ]:
# Võtab iga batchi (tekstid korpusest, kus laused on eraldatud \n sümboliga, 'text' : lause1 \n lause2 \n lause3)
# Tokenieerib laused ja lisab batchile labelid
# Väjund (x tähistab kõikide lausete arvu loetud batchi tekstides)
# {'input_ids': x*128*2 tensor, 'attention_mask': x*128 tensor, 'token_type_ids': x*128 tensor, 'labels': x*128*2 tensor}
def tokeniseeri_batch(batch):
    batch_laused = [lause for para in batch["text"] for lause in para.split("\n")]
    tokeniseeritud = tokenizer(batch_laused, max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")
    tokeniseeritud["labels"] = tokeniseeritud["input_ids"]
    return tokeniseeritud

In [ ]:
%%time
train_dataset_sisend = train_dataset.map(tokeniseeri_batch, batched=True, remove_columns=["text"])
train_dataset_sisend.set_format(type ='torch')

In [ ]:
train_dataset_sisend.to_json("treening_data.json")

In [12]:
treening_andmed = load_dataset("json", data_files={'train': ["korpus/test.json", "korpus/test2.json"]})["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:/Users/rauln/.cache/huggingface/datasets/json/default-14c4a4dc11d17ff8/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
with open("treening_data.tsv", 'w', newline='', encoding="utf-8") as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    for i in range(len(train_dataset_sisend)):
        writer.writerow(train_dataset_sisend[i])

In [22]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

In [21]:
config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

model = BertForMaskedLM(config)

training_args = TrainingArguments(
    output_dir='./train_results',
    per_device_train_batch_size=32,
    max_steps=10,
    learning_rate=1e-4,
    logging_dir='./train_logs',
    logging_steps=5, 
    save_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator = data_collator,
    train_dataset=treening_andmed
)

trainer.train()

NameError: name 'data_collator' is not defined

In [ ]:
import torch
from tqdm import tqdm
dataloader_test = torch.utils.data.DataLoader(train_dataset_sisend, batch_size = 16)
loop = tqdm(dataloader_test, leave = True)
for batch in loop:
    input_ids = batch["input_ids"]
    output = model(input_ids)

In [18]:
test_andmed = load_dataset("json", data_files={'test': ["korpus/test.json"]})["test"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:/Users/rauln/.cache/huggingface/datasets/json/default-86bf9a61ef87b506/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
test_andmed[:32]

{'input_ids': [[[2, 2],
   [13226, 1],
   [183, 34],
   [1545, 53],
   [1, 1],
   [1324, 30],
   [1465, 1],
   [1116, 34],
   [13226, 1],
   [698, 1],
   [685, 30],
   [62, 30],
   [1, 30],
   [8, 1],
   [18819, 30],
   [2818, 30],
   [1463, 30],
   [5, 1],
   [4671, 45],
   [3082, 35],
   [1596, 108],
   [24, 1],
   [495, 30],
   [3157, 49],
   [27580, 49],
   [6, 1],
   [3, 3],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
   [0, 0],
 

In [29]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    
    indeksid = np.where(labels[:, :, 0].flatten() != -100)[0]
    
    labels_lemma = labels[:, :, 0].flatten()[indeksid]
    labels_vorm = labels[:, :, 1].flatten()[indeksid]

    pred_lemma = np.take(np.argmax(pred[0], axis = 2).flatten(), indeksid)
    pred_vorm = np.take(np.argmax(pred[1], axis = 2).flatten(), indeksid)

    accuracy_lemma = accuracy_score(y_true=labels_lemma, y_pred=pred_lemma)
    accuracy_vorm = accuracy_score(y_true=labels_vorm, y_pred=pred_vorm)

    return {"accuracy_lemma": accuracy_lemma, "accuracy_vorm": accuracy_vorm}

model = BertForMaskedLM.from_pretrained("train_results\checkpoint-10")

trainer = Trainer(
    model=model,
    data_collator = data_collator,
    eval_dataset = test_andmed
)

trainer.evaluate()

loading configuration file train_results\checkpoint-10\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.25.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50005,
  "vocab_size_form": 111
}

loading weights file train_results\checkpoint-10\pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at train_results\checkpoint-10.
If y

{'eval_loss': 2.757899761199951,
 'eval_runtime': 16.7977,
 'eval_samples_per_second': 3.691,
 'eval_steps_per_second': 0.476}